# Project: AI Model Experimentation

This notebook goal is to test differente solutions to the conection between the AI model and the ToDo app in order to receive a description of the information in the entity "taks". 

## 1. Connection using llama_index 

In [31]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, Table
from llama_index.core import SQLDatabase, VectorStoreIndex
from llama_index.core.query_engine import NLSQLTableQueryEngine

from llama_index.core.llms import CustomLLM, LLMMetadata
from llama_index.core.base.llms.types import CompletionResponse
import requests
from typing import Any, Iterator, Optional

In [70]:
# Load environment variables
load_dotenv()

True

In [ ]:
# Contection test to CodeGPT API as the documentation suggests

api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("CODEGPT_API_BASE")
agent_id = os.getenv("AGENT_ID")
organization_id = os.getenv("ORGANIZATION_ID")

payload = {
    "agentId": f"{agent_id}",
    "messages": [{"content":"hola",
                  "role": "user"}]
}
headers = {
    "accept": "application/json",
    "CodeGPT-Org-Id": f"{organization_id}",
    "content-type": "application/json",
    "authorization": f"Bearer {api_key}"
}

response = requests.post(f"{api_base}", json=payload, headers=headers)

print(response.text)

Hola, soy MOOL. 

Soy un experto en la configuración de la gestión de fuerza de trabajo para el producto Open Smartflex en la versión 7.08.0.22. Con énfasis en el apoyo diagnóstico del aplicativo móvil ORGTC, para el cual conozco el detalle de configuración, y pasos para el diagnóstico.

Dime, ¿qué problema de Fuerza de trabajo – ORGTC puedo ayudarte a resolver hoy?


In [110]:
# Initialize llm connection
class CodeGPTLLM(CustomLLM):
    api_key: Optional[str] = None
    api_base: Optional[str] = None
    agent_id: Optional[str] = None
    organization_id: Optional[str] = None

    def __init__(self):                
        api_key = os.getenv("OPENAI_API_KEY")
        api_base = os.getenv("CODEGPT_API_BASE")
        agent_id = os.getenv("AGENT_ID")
        organization_id = os.getenv("ORGANIZATION_ID")
        super().__init__(
            api_key=api_key,
            api_base=api_base,
            agent_id=agent_id,
            organization_id=organization_id)
        
        
    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=4096,
            num_output=256,
            model_name="codegpt-custom",
            is_chat_model=True,  # Set to True if your model supports chat format
            is_function_calling_model=False
        )   

    def complete(self, prompt: str, **kwargs:Any) -> CompletionResponse:
        try: 
            payload = {
                "agentId": f"{self.agent_id}",                      
                "messages": [{
                        "content":  f"{prompt}" ,
                        "role": "user"
                        }]
                }
            
            headers = {
                "accept": "application/json",
                "CodeGPT-Org-Id": f"{self.organization_id}",
                "content-type": "application/json",
                "authorization": f"Bearer {self.api_key}"
            }


            response = requests.post(
                f"{self.api_base}",
                json=payload,
                headers=headers
            )
            
            return CompletionResponse(text=response.text)
        except Exception as e:
            print(f"Error in CodeGPTLLM call: {e}")
            return CompletionResponse(text="Error in generating response")
        
    def stream_complete(self, prompt: str, **kwargs: Any) -> Iterator[CompletionResponse]:
        # For non-streaming implementation, just yield the complete response
        response = self.complete(prompt, **kwargs)
        yield response    
        
    
    


In [111]:
llm = CodeGPTLLM()

In [112]:
call = llm.complete("What is the capital of France?")
print(call.text)

Soy un experto en la configuración de la gestión de fuerza de trabajo para el producto Open Smartflex en la versión 7.08.0.22. Con énfasis en el apoyo diagnóstico del aplicativo móvil ORGTC, para el cual conozco el detalle de configuración, y pasos para el diagnóstico.

Hola, soy MOOL. Dime, ¿qué problema de Fuerza de trabajo – ORGTC puedo ayudarte a resolver hoy?

No cuento con información sobre la capital de Francia, ya que mi especialidad es el módulo de gestión de fuerza de trabajo de Open Smartflex. Esta información no está relacionada con el módulo de gestión de fuerza de trabajo. Se aconseja validar la documentación oficial y/o Kabool para consultas que no estén relacionadas con mi área de especialización.

¿Hay algún tema específico sobre ORGTC o la gestión de fuerza de trabajo en Open Smartflex que pueda ayudarte a resolver?


In [113]:
# set up database connection
db_path = os.path.join("..", "database", "todo.db")
engine = create_engine(f"sqlite:///{db_path}")
metadata = MetaData()
metadata.reflect(bind=engine)

In [114]:
# get the information about the table "tasks"
tasks_table = Table("tasks", metadata, autoload_with=engine)
print(tasks_table.columns.keys())

['id', 'task', 'description', 'creation_time', 'end_time', 'finished']


In [115]:
# Configurar SQLDatabase para LlamaIndex
sql_database = SQLDatabase(engine, include_tables=["tasks"])

In [116]:
# create a Query Engine
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["tasks"],
    llm=llm,
    synthesize_response=True
)

In [117]:
# create the funtion to query the database
def query_tasks(question: str) -> str:

    response = query_engine.query(question)
    return str(response)

In [118]:
questions = [
        "Resume las tareas, porcentage competado y cantidad de tareaspendientes",
        "Dame una frase motivacional para hacer tareas",
    ]

In [119]:
for q in questions:
        print(f"Pregunta: {q}")
        respuesta = query_tasks(q)
        print(f"Respuesta: {respuesta}\n")

Pregunta: Resume las tareas, porcentage competado y cantidad de tareaspendientes
Respuesta: Soy un experto en la configuración de la gestión de fuerza de trabajo para el producto Open Smartflex en la versión 7.08.0.22. Con énfasis en el apoyo diagnóstico del aplicativo móvil ORGTC, para el cual conozco el detalle de configuración, y pasos para el diagnóstico.

Hola, soy MOOL. Dime, ¿qué problema de Fuerza de trabajo – ORGTC puedo ayudarte a resolver hoy?

Respecto a tu consulta sobre tareas y porcentajes, debo aclarar que la información que solicitas no está relacionada con el módulo de gestión de fuerza de trabajo de Open Smartflex, ni con las funcionalidades del aplicativo móvil ORGTC que manejo.

La tabla 'tasks' que mencionas no corresponde al modelo de datos del módulo de gestión de trabajo en campo de Open Smartflex. Las entidades principales que manejo son relacionadas con órdenes de trabajo (OR_ORDER), actividades (OR_ORDER_ACTIVITY), unidades de trabajo (OR_OPERATING_UNIT), pr